In [51]:
import pandas as pd
import pydirectinput as pdi
import time as t
import pyperclip
import pyautogui
import re

In [52]:
RES_PATTERN = r'(^|\b|\D|00000)(1 *[012] *\d *\d *\d *\d *\d)(\D|\b)'
RES_PATTERN2 = r'(\b|\D|00000)(1\s*[012]\s*(\d\s*){5})(\D)+((\d\s*){1,5}(( ?\. ?|,)\d\d?)?)(\D|\b)'

In [53]:
url = 'https://raw.githubusercontent.com/kvmilos/OREX/main/kontrahenci.csv'

In [54]:
dfs = pd.read_csv(url)
dic = dict(zip(dfs['kod'], dfs['pozycja']))

In [55]:
def slownik(rez, dic, dlugi=True) -> str:
    if rez != None:
        if not dlugi:
            if dic[int(rez)]:
                return str(dic[int(rez)])
            else:
                return 'Nie ma'
        else:
            if dic[int(rez)]:
                return '201-2-1-'+str(dic[int(rez)])
            else:
                return 'Nie ma'
    else:
        return 'Nie ma'

In [56]:
wb7 = pd.read_excel('wb7.xlsx')

In [57]:
wb7.head()

,Firma,Rachunek,Data księgowania,Data przyjęcia do realizacji,Strona transakcji,Rachunek strony transakcji,Tytuł,Kwota,Waluta,Typ operacji
0,OREX TRAVEL SP Z O O,91 1090 0004 0000 0012 7835 0001,2024-03-07,2024-03-07,OREX TRAVEL SP Z O O,76 1090 1841 0000 0001 5424 2475,Rezerwacja: 1246980 EIN11109000045747000001246980,-15.0,PLN,NaN
1,OREX TRAVEL SP Z O O,91 1090 0004 0000 0012 7835 0001,2024-03-07,2024-03-07,AGATA GMEREK KASPROWICZA 12 63-400 OSTRÓW WIEL...,09 1090 1160 0000 0001 0879 6007,Rezerwacja: 1246980,15.0,PLN,NaN
2,OREX TRAVEL SP Z O O,91 1090 0004 0000 0012 7835 0001,2024-03-07,2024-03-07,OREX TRAVEL SP Z O O,76 1090 1841 0000 0001 5424 2475,ANEX-REZ. 1243917 GOCZOK EIN681090000457470000...,-80.0,PLN,NaN
3,OREX TRAVEL SP Z O O,91 1090 0004 0000 0012 7835 0001,2024-03-07,2024-03-06,MIR-TOUR MIROSŁAWA JANKOWSKA-CZERMI ŃSKA UL. B...,66 1020 1778 0000 2602 0004 3224,ANEX-REZ. 1243917 GOCZOK,80.0,PLN,NaN
4,OREX TRAVEL SP Z O O,91 1090 0004 0000 0012 7835 0001,2024-03-07,2024-03-07,OREX TRAVEL SP Z O O,76 1090 1841 0000 0001 5424 2475,rez 1246564 Biernacka EIN881090000457470000012...,-40.0,PLN,NaN


In [58]:
wb7['Data księgowania'] = pd.to_datetime(wb7['Data księgowania'], format='%Y-%m-%d')
wb7['Data przyjęcia do realizacji'] = pd.to_datetime(wb7['Data przyjęcia do realizacji'], format='%Y-%m-%d')

In [61]:
def paste_text(text):
    t.sleep(0.2)
    pyperclip.copy(text)
    pyautogui.hotkey('ctrl', 'v')
    t.sleep(0.2)

In [62]:
def rozr():
    pdi.keyDown('alt')
    pdi.press('r')
    t.sleep(.2)
    pdi.press('t')
    t.sleep(.2) 
    pdi.press('z')
    t.sleep(.2)
    pdi.keyUp('alt')

In [67]:
def wpisz_rez(text):
    matches = re.findall(RES_PATTERN, text)
    if len(matches) == 1:
        paste_text(slownik(matches[0][1], dic))
        rozr()
    elif len(matches) > 1:
        wiele = re.findall(RES_PATTERN2, text)
        if len(wiele) >= 1:
            pdi.press('left')
            t.sleep(0.05)
            for rezerwacja in wiele:
                pdi.write(rezerwacja[4].replace(' ', ''))
                t.sleep(0.05)
                pdi.press('enter')
                t.sleep(0.05)
                pdi.write(slownik(rezerwacja[1].replace(' ', ''), dic))
                t.sleep(0.05)
                pdi.press('enter')
                t.sleep(0.05)
            for _ in range((len(wiele)-1)*2):
                pdi.keyDown('shift')
                pdi.press('tab')
                pdi.keyUp('shift')
                t.sleep(0.05)
            for _ in range((len(wiele)-1)):
                pdi.press('tab')
                t.sleep(0.05)
                pdi.press('tab')
                t.sleep(0.05)
                rozr()            
        else:
            paste_text('139-5')
            rozr()
    else:
        paste_text('139-5')
        rozr()

        

In [68]:

def perform_actions(row, nazwa):
    pdi.press('enter')
    t.sleep(0.05)
    pdi.press('tab')
    t.sleep(0.05)
    paste_text(nazwa)
    t.sleep(0.05)
    pdi.press('tab')
    t.sleep(0.05)
    paste_text(str(row['Tytuł']))
    t.sleep(0.05)
    pdi.press('tab')
    t.sleep(0.05)
    pdi.press('tab')
    t.sleep(0.05)
    pdi.write(str(row['Kwota']))
    t.sleep(0.05)
    pdi.press('tab')
    t.sleep(0.05)
    paste_text('130-19')
    t.sleep(0.05)
    pdi.press('tab')
    t.sleep(0.05)
    pdi.press('tab')
    t.sleep(0.05)
    wpisz_rez(row['Tytuł'])
    t.sleep(0.05)

In [73]:
# make df with only Data księgowania = XYZ
df = wb7[wb7['Data księgowania'] == '2024-02-19']
# delete rows where Strona transakcji = OREX TRAVEL SP Z O O
df = df[df['Strona transakcji'] != 'OREX TRAVEL SP Z O O']
# show sum of Kwota
df['Kwota'].sum()

132811.75

In [74]:
df.head()

,Firma,Rachunek,Data księgowania,Data przyjęcia do realizacji,Strona transakcji,Rachunek strony transakcji,Tytuł,Kwota,Waluta,Typ operacji
648,OREX TRAVEL SP Z O O,91 1090 0004 0000 0012 7835 0001,2024-02-19,2024-02-19,ARTUR KRZYSZTOF OHIRKO UL. WARECKA CZERSK 43A ...,91 1500 1272 1012 7004 0276 0000,nr rezerwacji1233559 Artur Ohirko,4310.00,PLN,NaN
649,OREX TRAVEL SP Z O O,91 1090 0004 0000 0012 7835 0001,2024-02-19,2024-02-19,MARZENA VIOLETTA BICZYSKO TEKLINÓW 138 98-400 ...,92 1090 1144 0000 0001 2217 2387,1233759 rezerwacja,836.25,PLN,NaN
650,OREX TRAVEL SP Z O O,91 1090 0004 0000 0012 7835 0001,2024-02-19,2024-02-19,ŚWIĘTY AGNIESZKA KATARZYNA 1 MAJA 34/6 44-206 ...,68 1050 1344 1000 0097 4997 0753,Numer Rezerwacji 1233457,4145.00,PLN,NaN
651,OREX TRAVEL SP Z O O,91 1090 0004 0000 0012 7835 0001,2024-02-19,2024-02-19,LESIUK IRMINA ul. GEN. JÓZEFAZAJĄCZKA 1/5/52 0...,81 1560 0013 2449 7554 0000 0003,Zaliczka I.Lesiuk i P.Kamiński nr.rez.1233177,770.25,PLN,NaN
652,OREX TRAVEL SP Z O O,91 1090 0004 0000 0012 7835 0001,2024-02-19,2024-02-19,WOJCIECH GÓRSKI UL. KRASZEWO 59 11-100 LIDZB...,56 1020 3570 0000 2102 0091 1081,REZERWACJA NUMER 1233500,7110.00,PLN,NaN


In [70]:
nazwa = 'Wyciąg bankowy za okres 19/02/2024'
print('Masz 5 sekund na otworzenie okna z wb')
t.sleep(5)
for _, row in df.iterrows():
    perform_actions(row, nazwa)

Masz 5 sekund na otworzenie okna z wb
